In [1]:
from pynq import Overlay
import asyncio
from psutil import cpu_percent

ol = Overlay("fact_intrpt.bit") #assumes you have the overlay bitstream in the current directory

In [ ]:
ol.ip_dict

# IP address map
You need to set up the interrupt controller in Vivado and get this information from there.

In [2]:
# IP's addresses
IP_CTRL = 0x00
AP_START = 0x1
GIER = 0x04
IP_IER = 0x08
IP_ISR = 0x0C
INTRPT_AP_DONE = 0x1
INTRPT_AP_READY = 0x2
OUT_REG = 0x10
INP_REG = 0x18

_INTRPT = INTRPT_AP_DONE 

fact_ip = ol.fact_intrpt
fact_ip.write(GIER, 0x1)
fact_ip.write(IP_IER, 0x3) #fix me

In [5]:
import asyncio
import time  
from datetime import datetime
import math

# You may want to comment out print statements in the read_ip and write_wait coroutines

# Coroutine that waits for an IP to be done.
async def read_ip(ip):
    while True:
        # Wait for the IP to finish.
        await ip.interrupt.wait()
        print('Interrupt received. Status register: {}'.format(ip.read(IP_ISR)))
        # Clear the interrupt and then print output's value.
        if (ip.read(IP_ISR) & INTRPT_AP_DONE):
            ip.write(IP_ISR, INTRPT_AP_DONE)
            print('interrupt received AP_DONE, out = {}'.format(ip.read(OUT_REG)))
        elif (ip.read(IP_ISR) & INTRPT_AP_READY):
            print('interrupt received AP_READY')

# Make a task from the read_ip coroutine and add it to the event loop
ip_task = asyncio.ensure_future(read_ip(fact_ip))
    
# Coroutine for writing input and starting the IP with delay
async def write_wait(interval):
    await asyncio.sleep(interval) #wait initial time
    factorial_input = 1
    #while True: #uncomment if you want to run forever
    for i in range(20): #uncomment if you want to run a limited number of times
        # write to input
        fact_ip.write(INP_REG, factorial_input) 
        currtime = time.time()
        fact_ip.write(IP_CTRL, AP_START) 
#         print("input {} started at time {}".format(factorial_input, currtime))
        await asyncio.sleep(interval)
        factorial_input += 1
         
# Run the event loop until the time interval expires
time_interval = .0001  # time in seconds
loop = asyncio.get_event_loop()
write_task = asyncio.ensure_future(write_wait(time_interval))

# Using psutil to record CPU utilization.
cpu_percent(percpu=True)  # Initializing the CPU monitoring.
loop.run_until_complete(write_task)
cpu_used = cpu_percent(percpu=True)

# Printing the CPU utilization
print('CPU Utilization = {cpu_used}'.format(**locals()))

# Removing the IP task from the event loop.
ip_task.cancel()

Interrupt received. Status register: 2
interrupt received AP_READY
Interrupt received. Status register: 2
interrupt received AP_READY
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 1
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 2
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 6
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 24
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 120
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 720
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 5040
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 40320
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 362880
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 3628800
Interrupt received. Status register: 3
interrupt received AP_DONE, out = 39916800
Interrupt r

True

CPU utilization returns percentage used on each core of the ARM

### Run factorial on CPU and see utilization

In [ ]:
import time  
from datetime import datetime
import math
from psutil import cpu_percent

cpu_percent(percpu=True)
for i in range(100):
    math.factorial(i)
cpu_used = cpu_percent(percpu=True)
# Printing the CPU utilization
print('CPU Utilization = {cpu_used}'.format(**locals()))  